# HW-5: Reaction Kinetics, Chemical Species, Non-Linear Least-Squares and Convolution Neural Networks 

## Import Libraries

In [ ]:
import numpy as np
print("Succesfully imported %s -- Version: %s"%(np.__name__,np.__version__))
import scipy
print("Succesfully imported %s -- Version: %s"%(scipy.__name__,scipy.__version__))
import matplotlib.pyplot as plt
print("Succesfully imported %s"%plt.__name__)
import pandas as pd
print("Succesfully imported %s -- Version: %s"%(pd.__name__,pd.__version__))
import sympy as sym 
print("Succesfully imported %s -- Version: %s"%(sym.__name__,sym.__version__))
from scipy import optimize
print("Succesfully imported %s"%optimize.__name__)
from scipy.optimize import fsolve
print("Succesfully imported %s"%fsolve.__name__)
from scipy.signal import find_peaks
print("Succesfully imported %s"%find_peaks.__name__)
from scipy.linalg import orth
print("Succesfully imported %s"%orth.__name__)
from scipy.integrate import odeint
print("Succesfully imported %s"%odeint.__name__)

## Section 1:/ Introduction

### Objectives 

The objective of this homework assignment is to estimate reaction parameters from measured species concentrations. We will consider the Langmuir-Hinshelwood reaction mechanism for a single reactans as a example. The concepts introduced in this homework, however, do carry over to other applications. Examples include the Brusselator model considered in HW-2, an auto-catalytic reaction or a combustion reactions with synthetic fuel. Various approaches to solve the inverse parameter estimation problem will be explored. We will explore derivative-free methods (requires computing norm of $M(p)$), derivative-based methods and convolutional neural network appproaches.

### Forward Model, Data and Inverse Model 

We will use the following three terms: forward model, data and inverse model. These terms are explained in the next paragraphs. 

The term forward model refers, loosely stated, to the numerical simulation model that mimmicks the underlying chemical system. The aim of solving the forward model is to arrive at a digital twin for the data collection process. In order to be more precise, we denote the state vector and the model parameter vector as $u$ and $p$, respectively. In our examples, the symbols $u$ and $p$ denote the vector of chemical species concentrations and the vector of reaction parameters, respectively. We will denote the time variable by $t$. For a given set of reaction parameters, the species concentration evolves over time. This implies that $u$ depends on both $t$ and $p$. We express this dependence by the notation $u(t,p)$. For a given vector $p$, the time evolution is described by a system of ordinary differential equations supplied with initial conditions $u_0$. The forward model can thus be expressed as: given a time interval $0 \leq t \leq T$, given a vector of reaction parameters $p$ and given a vector of initial conditions $u_0$, solve the system of ordinary differential equations 
\begin{equation}
\frac{d \, u}{dt} = f(u,p,t) \text{ for } 0 \leq t \leq T
  \text{ supplied with initial conditions } u(t=0) = u_0 
\end{equation}
to obtain $u(t,p)$. Techniques to solve this system of ordinary differential equations are explored in other homework assignments. The forward model assumes fixed parameter values $p$ when solving the system of ordinary differential equations for $u(t,p)$. The forward model can then be formaized as a mapping from the input $p$ to the output $u(t,p)$. After having solved the system of ordinary differential, the solution $u(t,p)$ is evaluated in a set of discrete time samples. In this way, a set of simulated values for $u(t,p)$ is found. In case that $n$ time samples are used, we have that $u$ is a vector of size $n$ or $u \in R^n$. The forward model is thus a mapping from $R^p$ to $R^n$.      

The term data refers to experimentally gathered measurements. We assume that data is collected in a discrete set of time instances ranging between $t=0$ and $t=T$. We will denote the measurements as $u^*$. We assume that the data is obtained from runnning forward model possibly perturbed by a noice term. This allows to experiment with various forms of measured data. We assume that the same time samples are used to construct both the measured $u^*$ and the simulated $u$ state values. Then both $u$ and $u^*$ are vectors of size $n$. 

The term inverse model refers to the process of determining the model parameters by combining the forward model and the data. In the inverse model, the model parameters $p$ are computed by solving a least squares problem. For arbitrary model parameters $p$, the output of the forward model differs from the measured values. This difference is also called model error or model discrepancy. It is denoted by $E(p)$ (as later its squared error is referred to as SSE. It is a vector of size $n$ thast is computed as 
\begin{equation}
  E(p) = u(p) - u^* \in R^n \, . 
\end{equation}
The magnitude (or squared norm) of $E(p)$ is computed as 
\begin{equation}
  SSE = \| E(p) \|^2 = \| u(p) - u^* \|^2 = \sum_{i=1}^n [ u_i(p) - u_i^* ]^2 
\end{equation}
The error vector $E(p)$ depends in general non-linearly on $p$. The objective in solving the inverse non-linear least-squares problem is to find a (local) minimizer of $E(p)$. For these parameter values, the forward model agrees best with the experimentally measured data. More formally, the objective is to find a set of parameters $p$ such that $E(p)$ has a (local) minimum in $p$. 

### Derivative-Free Optimization Methods 

Local search (simplex algorithm or Cobyla algorithm, see scipy.optimize): provide links to wiki for algorithm and to scipy.optimize for implementation details and usage.  

Global search (genetic algorithm, see scipy.optimize): as above. 

### Derivative-Based Optimization Methods with Numerically Computed Sensitivity 

Run derivative-based algorithm with numerically computed sensitivity. 

Least-Squares algorithm and general optimization algorithms (gradient descent and Newton type algorithms). 

### Forward Model Sensitivity 

The model error $E(p)$ is a mapping from $R^p$ to $R^n$ (in the same way the forward model is). The Jacobian of this mapping is the $n$-by-$m$ matrix defined as 
\begin{equation}
  \nabla_p E(p) = \left[ \partial E / \partial p_1, \ldots, \partial E / \partial p_m \right] \in R^{n \times m}
\end{equation}
(symbol $p$ attached to the nabla-operator to stress derivative wrt parameters $p$ is intended here). Given that the measured data is independent of $p$, this Jacobian can be equivalently written as 
\begin{equation}
  \nabla_p E(p) = \left[ \partial u / \partial p_1, \ldots, \partial u  / \partial p_m \right] \in R^{n \times m}
\end{equation}
The partial derivatives $\partial u / \partial p_j$ can be computed after having solved the forward model for $u(t,p)$. These partial derivatives are computed after having solved the system of ordinary differential equations for $u(t,p)$. 

For the computation of $\partial u / \partial p_j$, the automatic differentiation (reverse in time) implemented in the library [JAX](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html). An example for a pendulumn is given in [this example](https://github.com/google/jax/pull/2817). Another example is given in [Deep Implicit Layers](http://implicit-layers-tutorial.org/implicit_functions/) (using JAX, extension of Numpy, autograd outdated). An example of computing the sensitivities $\partial u / \partial p_j$ is given in Cantera, as shown in this example [sensivity computation in Cantera](https://cantera.org/examples/python/reactors/sensitivity1.py.html). 

### Derivative-Based Optimization Methods

Run derivative-based algorithm sensitivity computed using automatic-differentiation. Reuse same derivative-based methods as above. 

### Convolutional Neural Network Models 

The example [blog page on the use of fluxdiffeq](https://julialang.org/blog/2019/01/fluxdiffeq/) will be used as a demonstration of what is currently possible (despite it being develop in Julia). 

### Overview of This Assignment 

In this homework assignment, we intend to go through the following steps: 

## Section 2/: Example from Catalytic Reactions from Alvirez-Raminez-2016 

### Section 1.2/: Forward Model 
Describe the forward model as ODE both transient and quasi-state approximation. Describe the input-output relation of the forward model, describe the state $u$ and the parameters $p$. Describe how to solve the forward model numerically. Describe what the reference solutions are. 

Describe forward model Eqn. (17) in Alvirez-Raminez-2016; set-up forward model possibly using chempy; solve the forward model using scipy.integrate.solve_ivp() and compare solution with Fig. 1 ($\Sigma_T=6.0$) and Fig. 2 ($\Sigma_T=15.0$) in Alvirez-Raminez-2016;

### Section 2.2:/ Available Data
Describe what data is avaiable (either from lab or literature).

### Section 3.3:/ Combining Model and Data to Infer Information 
Describe how to perform linear or non-linear regression to find the parameters.  

Formulate cost functional as Eqn. (43) in Alvirez-Raminez-2016 (sum of various time samples of single concentration profile). 

 ## Section 2:/ Forward Model: Time-Integration of a System of Coupled ODEs

## Section 3/: Forward Model: Sensitivity of Solution wrt Model Parameters 

## Section 4/: Inverse Model Formulation and Sensivity 

Describe least-squares fit between model and measured data. The data can be output of the forward model perturbed eventually by noice. Describe gradient of least-squares functional wrt model parameters. Use upper and lower bounds on the design variables as the easier way to regularize the problem. 

## Section 6/:  Inverse Model Solve using Convolutional Neural Networks 

Training of network. Use network to solve non-linear least-squares problem. 

## References  

1. [wiki on autocatalysis](https://en.wikipedia.org/wiki/Autocatalysis) 
2. [wiki on non-linear least-squares](https://en.wikipedia.org/wiki/Non-linear_least_squares)
3. [wiki on Levenberg-Marquard algorithm](https://en.wikipedia.org/wiki/Levenberg–Marquardt_algorithm)  
4. examples of non-linear least-squares problem with ODE constraint: [example](https://stackoverflow.com/questions/11278836/fitting-data-to-system-of-odes-using-python-via-scipy-numpy)
5. [Cantera for chemical reactiions](https://cantera.org) 
6. not-Python: [blog page on the use of fluxdiffeq](https://julialang.org/blog/2019/01/fluxdiffeq/)
7. not-Python: [Example of a non-linear least squares problem ready to roll](https://julialang.org/blog/2019/01/fluxdiffeq/) Non-Linear least-squares problem using a neural network problem.

In [1]:
import jax 

RuntimeError: jaxlib is version 0.1.69, but this version of jax requires version >= 0.3.7.

In [2]:
from functools import partial
from jax.experimental.ode import odeint
import jax.numpy as jnp

def f(state, t, rho, sigma, beta):
  x, y, z = state
  return jnp.array([sigma * (y - x), x * (rho - z) - y, x * y - beta * z])

ys = odeint(partial(f, rho=28., sigma=10., beta=8./3),
            y0=jnp.array([1., 1., 1.]),
            t=jnp.linspace(0, 10., 10000))

AttributeError: module 'jax' has no attribute 'version'